<a href="https://colab.research.google.com/github/tariman67/TestAddIn/blob/master/test_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# نصب پکیج های ضروری برای Starling، Qwen و Deepseek
!pip install accelerate flash-attn transformers_stream_vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement transformers_stream_vllm (from versions: none)
ERROR: No matching distribution found for transformers_stream_vllm


In [4]:
# نصب پکیج های ضروری برای Qwen، Llama و Deepseek
!pip install tiktoken accelerate

In [6]:
from huggingface_hub import notebook_login
# این تابع توکن شما را از سکرتز فراخوانی و فعال می‌کند.
notebook_login()

In [1]:
# نصب کتابخانه های اصلی ترنسفورمرز، تورچ و سابورد-انکودینگ
# نصب کتابخانه های اصلی و وابستگی های مورد نیاز برای مدل های جدید
!pip install transformers torch sentencepiece accelerate protobuf bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login

# اگر توکن (HF Token) خود را در محیط کولب ذخیره نکرده‌اید، این تابع را اجرا کنید.
# پس از اجرا، یک پنجره باز می‌شود.
# توکن Hugging Face خود را که قبلاً از settings/Access Tokens کپی کرده‌اید، وارد کنید.
login()

In [3]:
from transformers import AutoTokenizer
import sys

# تنظیم حداکثر اندازه بازگشتی برای رفع خطاهای احتمالی در مدل های بزرگ
sys.setrecursionlimit(4000)

# --- ۱. تعریف متن بسیار طولانی‌تر و تخصصی فارسی ---
# این متن طولانی، مقایسه کارایی واقعی توکن‌سازی را بهتر نشان می‌دهد.
farsi_text_long = """
مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.
"""

# --- ۲. لیست جامع مدل‌های کاندید ---
models_to_test = {
    # مدل های Qwen
    "Qwen-3-14B-Chat": "Qwen/Qwen2-14B-Instruct",
    "Qwen-3-7B-Chat": "Qwen/Qwen2-7B-Instruct",

    # مدل های Llama (احراز هویت شده)
    "Llama-3.1-8B-Instruct": "meta-llama/Meta-Llama-3.1-8B-Instruct",

    # مدل های Gemma (احراز هویت شده)
    "Gemma-2-9B-It": "google/gemma-2-9b-it",

    # مدل های Mistral
    "Mistral-7B-Instruct-v0.2": "mistralai/Mistral-7B-Instruct-v0.2",

    # مدل های Deepseek
    "DeepSeek-R-8B-Base": "deepseek-ai/deepseek-r-8b",

    # مدل های Command R
    "C4AI Command-R7B": "CohereForAI/c4ai-command-r7b-12-2024",
    "C4AI Command-R7B-Arabic": "CohereForAI/c4ai-command-r7b-arabic-02-2025",

    # مدل Phi
    "Phi-3-medium-14B-4K-Instruct": "microsoft/Phi-3-medium-4k-instruct",
}

print(f"--- متن آزمایشی (طول متن: {len(farsi_text_long)} کاراکتر) --- \n{farsi_text_long}\n--------------------------\n")

# --- ۳. اجرای تست و جمع‌آوری نتایج ---
results = {}
for name, model_id in models_to_test.items():
    print(f"در حال تست: {name}...")
    try:
        # تنظیمات خاص برای بارگذاری توکنایزر
        if "Qwen" in name or "DeepSeek" in name or "Phi-3" in name:
            tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        else:
            tokenizer = AutoTokenizer.from_pretrained(model_id)

        tokens = tokenizer.tokenize(farsi_text_long)
        token_count = len(tokens)

        results[name] = token_count
        print(f"✅ نتیجه: {token_count} توکن.")

    except Exception as e:
        results[name] = "❌ خطا"
        print(f"❌ مدل {name} با خطا مواجه شد. (ممکن است مشکل از دسترسی یا وابستگی باشد.)")


# --- ۴. نمایش نتایج نهایی ---
print("\n\n--- جدول خلاصه نتایج نهایی (مرتب شده بر اساس بهترین کارایی) ---")
# نتایج بر اساس کمترین تعداد توکن مرتب می‌شوند
final_sorted_results = sorted(
    [item for item in results.items() if isinstance(item[1], int)],
    key=lambda item: item[1]
)

for rank, (name, count) in enumerate(final_sorted_results):
    print(f"رتبه {rank + 1}: {count} توکن — {name}")

--- متن آزمایشی (طول متن: 740 کاراکتر) --- 

مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.

--------------------------

در حال تست: Qwen-3-14B-Chat...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


❌ مدل Qwen-3-14B-Chat با خطا مواجه شد. (ممکن است مشکل از دسترسی یا وابستگی باشد.)
در حال تست: Qwen-3-7B-Chat...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

✅ نتیجه: 741 توکن.


--- جدول خلاصه نتایج نهایی (مرتب شده بر اساس بهترین کارایی) ---
رتبه 1: 229 توکن — Gemma-2-9B-It
رتبه 2: 239 توکن — C4AI Command-R7B-Arabic
رتبه 3: 412 توکن — Qwen-3-7B-Chat
رتبه 4: 718 توکن — Mistral-7B-Instruct-v0.2
رتبه 5: 741 توکن — Phi-3-medium-14B-4K-Instruct


In [1]:
from transformers import AutoTokenizer
import sys

# تنظیم حداکثر اندازه بازگشتی برای رفع خطاهای احتمالی در مدل های بزرگ
sys.setrecursionlimit(4000)

# --- ۱. تعریف متن بسیار طولانی‌تر و تخصصی فارسی ---
# این متن طولانی، مقایسه کارایی واقعی توکن‌سازی را بهتر نشان می‌دهد.
farsi_text_long = """
مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.
"""

# --- ۲. لیست جامع مدل‌های کاندید ---
models_to_test = {
    # مدل های Qwen
    "Qwen-3-14B-Chat": "Qwen/Qwen2-14B-Instruct",
    "Qwen-3-7B-Chat": "Qwen/Qwen2-7B-Instruct",

    # مدل های Llama
    "Llama-3.1-8B-Instruct": "meta-llama/Meta-Llama-3.1-8B-Instruct",

    # مدل های Gemma
    "Gemma-2-9B-It": "google/gemma-2-9b-it",
    "CodeGemma-7B-Instruct (New)": "google/codegemma-7b-it",

    # مدل های Mistral
    "Mistral-7B-Instruct-v0.2": "mistralai/Mistral-7B-Instruct-v0.2",

    # مدل های Deepseek
    "DeepSeek-R-8B-Base": "deepseek-ai/deepseek-r-8b",

    # مدل های Command R
    "C4AI Command-R7B": "CohereForAI/c4ai-command-r7b-12-2024",
    "C4AI Command-R7B-Arabic": "CohereForAI/c4ai-command-r7b-arabic-02-2025",

    # مدل Phi
    "Phi-3-medium-14B-4K-Instruct": "microsoft/Phi-3-medium-4k-instruct",

    # مدل Starling (New)
    "Starling-LM-7B-Beta": "HuggingFaceH4/Starling-LM-7B-beta",
}

print(f"--- متن آزمایشی (طول متن: {len(farsi_text_long)} کاراکتر) --- \n{farsi_text_long}\n--------------------------\n")

# --- ۳. اجرای تست و جمع‌آوری نتایج ---
results = {}
for name, model_id in models_to_test.items():
    print(f"در حال تست: {name}...")
    try:
        # تنظیمات خاص برای بارگذاری توکنایزر
        if "Qwen" in name or "DeepSeek" in name or "Phi-3" in name or "Starling" in name:
            tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        else:
            tokenizer = AutoTokenizer.from_pretrained(model_id)

        tokens = tokenizer.tokenize(farsi_text_long)
        token_count = len(tokens)

        results[name] = token_count
        print(f"✅ نتیجه: {token_count} توکن.")

    except Exception as e:
        results[name] = "❌ خطا"
        print(f"❌ مدل {name} با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)")


# --- ۴. نمایش نتایج نهایی ---
print("\n\n--- جدول خلاصه نتایج نهایی (مرتب شده بر اساس بهترین کارایی) ---")
final_sorted_results = sorted(
    [item for item in results.items() if isinstance(item[1], int)],
    key=lambda item: item[1]
)

for rank, (name, count) in enumerate(final_sorted_results):
    print(f"رتبه {rank + 1}: {count} توکن — {name}")

--- متن آزمایشی (طول متن: 740 کاراکتر) --- 

مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.

--------------------------

در حال تست: Qwen-3-14B-Chat...
❌ مدل Qwen-3-14B-Chat با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)
در حال تست: Qwen-3-7B-Chat...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

✅ نتیجه: 741 توکن.
در حال تست: Starling-LM-7B-Beta...
❌ مدل Starling-LM-7B-Beta با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)


--- جدول خلاصه نتایج نهایی (مرتب شده بر اساس بهترین کارایی) ---
رتبه 1: 229 توکن — Gemma-2-9B-It
رتبه 2: 229 توکن — CodeGemma-7B-Instruct (New)
رتبه 3: 239 توکن — C4AI Command-R7B
رتبه 4: 239 توکن — C4AI Command-R7B-Arabic
رتبه 5: 246 توکن — Llama-3.1-8B-Instruct
رتبه 6: 412 توکن — Qwen-3-7B-Chat
رتبه 7: 718 توکن — Mistral-7B-Instruct-v0.2
رتبه 8: 741 توکن — Phi-3-medium-14B-4K-Instruct


In [3]:
from transformers import AutoTokenizer
import sys

# تنظیم حداکثر اندازه بازگشتی برای رفع خطاهای احتمالی در مدل های بزرگ
sys.setrecursionlimit(4000)

# --- ۱. تعریف متن بسیار طولانی‌تر و تخصصی فارسی + عامیانه ---
farsi_text_long = """
مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.

حالا بیا صادق باشیم، آقا این آسانسوره ما هی میاد و می‌ره، صدای عجیب غریب میده، می‌ترسم واقعاً وسطش گیر کنم. مگه اون نصاب قبلیه نگفته بود این آسانسور شش نفر ظرفیت داره؟ پس چرا دو تا آدم وایمیستیم توش، می‌لرزه و سنسورهاش خطا میده؟ کلاً اون گواهی استانداردی که شرکت به ما داده بود، معتبره یا فقط یه برگه کاغذ الکیه؟
می‌خوام مطمئن شم دیگه وسط کار ایراد فنی نگیریم و همه چیز طبق قانون پیش بره. یعنی اگه یه قطعه‌ای رو الان بذاریم، فردا اداره استاندارد گیر نده و نگن باید عوض بشه؟ خلاصه بگم، ما می‌خوایم یه آسانسور توپ، مطمئن و بی‌خطر داشته باشیم که همه‌مون راحت باشیم و البته هزینه‌شم زیاد نشه. الان باید به این مهندس ناظر چی بگم که خیال همه راحت بشه و این لرزش‌ها تموم بشه؟ فکر کنم اون قطعه «فلان» که گفتین، از برند «بهمان» نیست. چه‌جوری چک کنم؟
"""

# --- ۲. لیست جامع مدل‌های کاندید (به روز شده) ---
models_to_test = {
    # مدل های Qwen
    "Qwen-3-14B-Chat": "Qwen/Qwen2-14B-Instruct",
    "Qwen-3-7B-Chat": "Qwen/Qwen2-7B-Instruct",

    # مدل های Llama
    "Llama-3.1-8B-Instruct": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "Mixtral-8x7B-Instruct-v0.1 (MoE)": "mistralai/Mixtral-8x7B-Instruct-v0.1", # جدید

    # مدل های Gemma (خانواده کامل)
    "Gemma-2-9B-It": "google/gemma-2-9b-it",
    "Gemma-2-9B (Base)": "google/gemma-2-9b", # جدید
    "CodeGemma-7B-Instruct": "google/codegemma-7b-it",

    # مدل های Mistral/Falcon
    "Mistral-7B-Instruct-v0.2": "mistralai/Mistral-7B-Instruct-v0.2",
    "Falcon-7B-Instruct": "tiiuae/falcon-7b-instruct", # جدید

    # مدل های Deepseek/Command R
    "DeepSeek-R-8B-Base": "deepseek-ai/deepseek-r-8b",
    "C4AI Command-R7B": "CohereForAI/c4ai-command-r7b-12-2024",
    "C4AI Command-R7B-Arabic": "CohereForAI/c4ai-command-r7b-arabic-02-2025",

    # مدل Phi/Starling
    "Phi-3-medium-14B-4K-Instruct": "microsoft/Phi-3-medium-4k-instruct",
    "Starling-LM-7B-Beta": "HuggingFaceH4/Starling-LM-7B-beta",
}

print(f"--- متن آزمایشی (طول متن: {len(farsi_text_long)} کاراکتر) --- \n{farsi_text_long}\n--------------------------\n")

# --- ۳. اجرای تست و جمع‌آوری نتایج ---
results = {}
for name, model_id in models_to_test.items():
    print(f"در حال تست: {name}...")
    try:
        # تنظیمات خاص برای بارگذاری توکنایزر
        needs_trust = any(x in name for x in ["Qwen", "DeepSeek", "Starling", "Falcon"])

        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=needs_trust)

        tokens = tokenizer.tokenize(farsi_text_long)
        token_count = len(tokens)

        results[name] = token_count
        print(f"✅ نتیجه: {token_count} توکن.")

    except Exception as e:
        results[name] = "❌ خطا"
        print(f"❌ مدل {name} با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)")


# --- ۴. نمایش نتایج نهایی ---
print("\n\n--- جدول خلاصه نتایج نهایی (مرتب شده بر اساس بهترین کارایی) ---")
final_sorted_results = sorted(
    [item for item in results.items() if isinstance(item[1], int)],
    key=lambda item: item[1]
)

for rank, (name, count) in enumerate(final_sorted_results):
    print(f"رتبه {rank + 1}: {count} توکن — {name}")

--- متن آزمایشی (طول متن: 1482 کاراکتر) --- 

مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.

حالا بیا صادق باشیم، آقا این آسانسوره ما هی میاد و می‌ره، صدای عجیب غریب میده، می‌ترسم واقعاً وسطش گیر کنم. مگه اون نصاب قبلیه نگفته بود این آسانسور شش نفر ظرفیت داره؟ پس چرا دو تا آدم وایمیستیم توش، می‌لرزه و سنسو

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

✅ نتیجه: 1477 توکن.
در حال تست: Starling-LM-7B-Beta...
❌ مدل Starling-LM-7B-Beta با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)


--- جدول خلاصه نتایج نهایی (مرتب شده بر اساس بهترین کارایی) ---
رتبه 1: 493 توکن — Gemma-2-9B-It
رتبه 2: 493 توکن — Gemma-2-9B (Base)
رتبه 3: 493 توکن — CodeGemma-7B-Instruct
رتبه 4: 541 توکن — Llama-3.1-8B-Instruct
رتبه 5: 543 توکن — C4AI Command-R7B
رتبه 6: 543 توکن — C4AI Command-R7B-Arabic
رتبه 7: 862 توکن — Qwen-3-7B-Chat
رتبه 8: 1433 توکن — Mixtral-8x7B-Instruct-v0.1 (MoE)
رتبه 9: 1433 توکن — Mistral-7B-Instruct-v0.2
رتبه 10: 1477 توکن — Phi-3-medium-14B-4K-Instruct
رتبه 11: 1634 توکن — Falcon-7B-Instruct


In [7]:
from transformers import AutoTokenizer
import sys

# تنظیم حداکثر اندازه بازگشتی برای رفع خطاهای احتمالی
sys.setrecursionlimit(4000)

# --- ۱. تعریف متن‌های آزمایشی ---
farsi_conversation_text = """
مطابق الزامات مبحث پانزدهم مقررات ملی ساختمان، حداقل ظرفیت کابین آسانسور برای یک ساختمان مسکونی با هشت طبقه توقف، ۶ نفر یا ۴۵۰ کیلوگرم با سرعت نامی یک متر بر ثانیه و ارتفاع مسیر صعود ۳۰ متر در نظر گرفته می‌شود.
در صورت عدم رعایت استانداردهای فنی و نقشه‌های اجرایی مصوب سازمان نظام مهندسی، مجری موظف به اجرای مجدد کلیه قسمت‌های مغایر با مقررات است. تمامی تغییرات در مشخصات فنی باید کتباً به ناظر اعلام و پس از تأیید نهایی، اعمال گردند. رعایت دقیق فواصل ایمنی و محاسبات بار دینامیکی، از اصول کلیدی تضمین‌کننده عملکرد صحیح و ایمنی دستگاه است.
عدم انطباق با این الزامات، منجر به ابطال گواهی ایمنی و توقف بهره‌برداری از آسانسور خواهد شد. مهندسین ناظر، مسئولیت مستقیم بررسی گواهی استاندارد قطعات و مطابقت آن‌ها با برندهای مجاز را بر عهده دارند.

حالا بیا صادق باشیم، آقا این آسانسوره ما هی میاد و می‌ره، صدای عجیب غریب میده. می‌خوام مطمئن شم دیگه وسط کار ایراد فنی نگیریم. قیمت این قطعه اصلی، مثلاً موتور فلان، چنده؟ میشه یه تخفیف خوب بگیریم؟ بالاخره ما حجم کارمون زیاده و این همه هزینه کردیم. اگه قیمت رو بیارید پایین، همه‌مون راحت‌تریم. این قیمتی که شما زدین، خیلی بالاتره از اون چیزی که رقیبتون به ما گفته بود!
"""

json_output_text = """
{
  "status": "Success",
  "data": {
    "parameters": [
      {
        "name": "MotorPower_KW",
        "value": 7.5
      },
      {
        "name": "RatedSpeed_M_Per_S",
        "value": 1.0
      }
    ],
    "recommendation": {
      "Capacity": "6_person",
      "DynamicLoad_KG": 450,
      "Compliance": "Mabhas_15_OK"
    }
  }
}
"""

# --- ۲. لیست جامع مدل‌های کاندید (به روز شده) ---
models_to_test = {
    # مدل های Qwen
    "Qwen-3-14B-Chat": "Qwen/Qwen2-14B-Instruct",
    "Qwen-3-7B-Chat": "Qwen/Qwen2-7B-Instruct",

    # مدل های Llama
    "Llama-3.1-8B-Instruct": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "Llama-3-8B-Instruct (New)": "meta-llama/Llama-3-8B-Instruct",
    "Mixtral-8x7B-Instruct-v0.1 (MoE)": "mistralai/Mixtral-8x7B-Instruct-v0.1",

    # مدل های Gemma
    "Gemma-2-9B-It": "google/gemma-2-9b-it",
    "Gemma-2-9B (Base)": "google/gemma-2-9b",
    "CodeGemma-7B-Instruct": "google/codegemma-7b-it",

    # مدل های Deepseek/Microsoft
    "DeepSeek-R-8B-Base": "deepseek-ai/deepseek-r-8b",
    "DeepSeek-7B-Chat (New)": "deepseek-ai/deepseek-7b-chat",
    "Phi-3-medium-14B-4K-Instruct": "microsoft/Phi-3-medium-4k-instruct",

    # مدل های Command R/Falcon
    "C4AI Command-R7B": "CohereForAI/c4ai-command-r7b-12-2024",
    "C4AI Command-R7B-Arabic": "CohereForAI/c4ai-command-r7b-arabic-02-2025",
    "Falcon-7B-Instruct": "tiiuae/falcon-7b-instruct",
}

print("--- تست نهایی در دو بخش (مکالمه فارسی و خروجی JSON) در حال اجراست. ---")

# --- ۳. اجرای تست و جمع‌آوری نتایج ---
results_conversation = {}
results_json = {}

for name, model_id in models_to_test.items():
    print(f"\nدر حال تست: {name}...")
    try:
        # تنظیمات خاص برای بارگذاری توکنایزر
        needs_trust = any(x in name for x in ["Qwen", "DeepSeek", "Falcon", "Phi"])

        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=needs_trust)

        # --- تست ۱: مکالمه فارسی ---
        tokens_conv = tokenizer.tokenize(farsi_conversation_text)
        results_conversation[name] = len(tokens_conv)

        # --- تست ۲: خروجی JSON ---
        tokens_json = tokenizer.tokenize(json_output_text)
        results_json[name] = len(tokens_json)

        print(f"✅ موفقیت. (مکالمه: {len(tokens_conv)} توکن, JSON: {len(tokens_json)} توکن)")

    except Exception as e:
        results_conversation[name] = "❌ خطا"
        results_json[name] = "❌ خطا"
        print(f"❌ مدل {name} با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)")


# --- ۴. نمایش نتایج نهایی ---
# --- نتایج مکالمه فارسی ---
print("\n\n--- جدول خلاصه نتایج نهایی (۱) کارایی در مکالمه فارسی (فنی + عامیانه) ---")
final_sorted_conv = sorted(
    [item for item in results_conversation.items() if isinstance(item[1], int)],
    key=lambda item: item[1]
)
for rank, (name, count) in enumerate(final_sorted_conv):
    print(f"رتبه {rank + 1}: {count} توکن — {name}")

print("\n--------------------------")

# --- نتایج JSON ---
print("\n--- جدول خلاصه نتایج نهایی (۲) کارایی در تولید خروجی JSON (حداقل توکن) ---")
final_sorted_json = sorted(
    [item for item in results_json.items() if isinstance(item[1], int)],
    key=lambda item: item[1]
)
for rank, (name, count) in enumerate(final_sorted_json):
    print(f"رتبه {rank + 1}: {count} توکن — {name}")

--- تست نهایی در دو بخش (مکالمه فارسی و خروجی JSON) در حال اجراست. ---

در حال تست: Qwen-3-14B-Chat...
❌ مدل Qwen-3-14B-Chat با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)

در حال تست: Qwen-3-7B-Chat...
✅ موفقیت. (مکالمه: 637 توکن, JSON: 113 توکن)

در حال تست: Llama-3.1-8B-Instruct...
✅ موفقیت. (مکالمه: 387 توکن, JSON: 110 توکن)

در حال تست: Llama-3-8B-Instruct (New)...
❌ مدل Llama-3-8B-Instruct (New) با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)

در حال تست: Mixtral-8x7B-Instruct-v0.1 (MoE)...
✅ موفقیت. (مکالمه: 1073 توکن, JSON: 145 توکن)

در حال تست: Gemma-2-9B-It...
✅ موفقیت. (مکالمه: 357 توکن, JSON: 133 توکن)

در حال تست: Gemma-2-9B (Base)...
✅ موفقیت. (مکالمه: 357 توکن, JSON: 133 توکن)

در حال تست: CodeGemma-7B-Instruct...
✅ موفقیت. (مکالمه: 357 توکن, JSON: 133 توکن)

در حال تست: DeepSeek-R-8B-Base...
❌ مدل DeepSeek-R-8B-Base با خطا مواجه شد. (احتمالاً مشکل از دسترسی، وابستگی یا مجوز است.)

در حال تست: DeepSeek-7B-Chat (New)...
❌ مدل DeepSeek-7